# Confusion matrix for CocoAPI

The "main_Evaluate_Object_Detection.ipynb" uses the CocoApi to evaluate the model performance, e.g., AP50. But that code does not include the output of confusion matrix (e.g., TP, FP, and FN).

This script provides the output of confusion matrix using **FiftyOne** tool

FiftyOne’s implementation of COCO-style evaluation matches the reference implementation available via pycocotools (https://github.com/cocodataset/cocoapi).

Refer to: 
https://docs.voxel51.com/integrations/coco.html#loading-coco-formatted-data

## Install fiftyone and cocoapi

In [410]:
#   For windows:
#        pip3 install fiftyone --user

#   For windows:
#        install pycocotools-2.0:
#        pip install git+https://github.com/philferriere/cocoapi.git#egg=pycocotools^&subdirectory=PythonAPI

## Load coco dataset (ground-truth)

**Note**: 

(1) the folder/path of images in json file must contains " / " instead of " \ " 

(2) In json file, "image_id" and the "id" for annotations must not start "0", otherwise the results will be not correct

In [411]:
import fiftyone as fo

# A name for the dataset
# name = "my-dataset"

# GJO test dataset
image_path = r"U:/AIMMW/Tianlong/Peng/Paper_SSL/Dataset/labeled_data/subsets/Test/test"
labels_path = r"U:/AIMMW/Tianlong/Peng/Paper_SSL/Dataset/labeled_data/subsets/Test/test.json"

# The type of the dataset being imported
dataset_type = fo.types.COCODetectionDataset  # coco format

dataset = fo.Dataset.from_dir(
    dataset_type=dataset_type,
    data_path=image_path,
    labels_path=labels_path,
    # include_id=True,
    # name=name,
)
# print(dataset)

 100% |███████████████████| 29/29 [1.8s elapsed, 0s remaining, 16.2 samples/s]      


In [412]:
# show the first data in the dataset,
# Carefully check the "label" name!
# Note the coordinates of the bbox is scaled into [0,1]

# print(dataset.first().detections.detections[0])

## Load predictions

When inferencing a model on a test dataset using Detectron2, it will output a prediction file named "coco_instances_results.json" with predicted bbox

Note: when load predictions in a json file, the "id" in "images" field must start from **"1"**, Otherwise it will not work. You may change the "id in json using: *https://github.com/TianlongJia/Tools/tree/master/Vision/Manage_pic_and_folder//Modify_JSON.ipynb*

In [413]:
import fiftyone.utils.coco as fouc
import json

#
# Mock COCO predictions, where:
# - `image_id` corresponds to the `coco_id` field of `coco_dataset`
# - `category_id` corresponds to classes in `coco_dataset.default_classes`
#
# predictions = [
#     {"image_id": 1, "category_id": 18, "bbox": [258, 41, 348, 243], "score": 0.87},
#     {"image_id": 2, "category_id": 11, "bbox": [61, 22, 504, 609], "score": 0.95},
# ]

json_file= r"C:\Users\tjian\Desktop\test_results\coco_instances_results.json"

with open(json_file, 'r') as fcc_file:
    fcc_data = json.load(fcc_file)
# print(fcc_data)
    
predictions = fcc_data


# Add COCO predictions to `predictions` field of dataset
classes = dataset.default_classes
# classes = ['hold_space', 'entrap bean', 'free bean',]
print("classes: ", classes)

fouc.add_coco_labels(dataset, 
                    #  coco_id_field="detections", 
                     label_field="predictions", 
                     labels_or_path=predictions, 
                     classes=classes)

# # Verify that predictions were added to two images
print(dataset.count("predictions"))  # 2

classes:  ['0', 'entrap bean', 'free bean']
29


In [414]:
# dataset.values([dataset.detections, "id"])

In [415]:
# print(dataset.count("detections"))  # the number of images

In [416]:
# show the first data (with prediction bbox) in the dataset ,
# Carefully check the "label" name!
# Note the coordinates of the predicted bbox is scaled into [0,1]

# print(dataset.first())

In [417]:
# sample = dataset.first()
# print(sample.detections.detections[0])

## Output results 

by comparing ground-truth labels and predictions bbox

**Note**: Predicted and ground truth objects are matched using a specified IoU threshold (default = 0.50). This threshold can be customized via the iou parameter. The IoU selection affects the calculation of TP, FN, are FP value.

In [418]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

# Evaluate the objects in the `predictions` field with respect to the
# objects in the `ground_truth` field
results = dataset.evaluate_detections(
    "predictions",
    gt_field="detections",
    iou=0.5,
    method="coco",   
    eval_key="eval",
    compute_mAP=True,
)

Evaluating detections...
 100% |███████████████████| 29/29 [487.2ms elapsed, 0s remaining, 59.5 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 29/29 [975.4ms elapsed, 0s remaining, 29.7 samples/s]      


### mAP

Note: this is the mAP at IoU=.50:.05:.95

In [419]:
print(results.mAP())

0.30267943877876013


### TP, FP, FN

In [420]:
# Print some statistics about the total TP/FP/FN counts
print("TP: %d" % dataset.sum("eval_tp"))
print("FP: %d" % dataset.sum("eval_fp"))
print("FN: %d" % dataset.sum("eval_fn"))

TP: 46
FP: 35
FN: 17


### Precision, recall, F1-score

In [421]:
classes = 2
counts = dataset.count_values("detections.detections.label")
classes = sorted(counts, key=counts.get, reverse=True)[:classes]

# Print a classification report
results.print_report(classes=classes)

              precision    recall  f1-score   support

   free bean       0.62      0.92      0.74        39
 entrap bean       0.43      0.42      0.43        24

   micro avg       0.57      0.73      0.64        63
   macro avg       0.53      0.67      0.58        63
weighted avg       0.55      0.73      0.62        63



### Precision-recall curve

In [ ]:
plot = results.plot_pr_curves(classes=classes)
plot.show()

### Confusion matrix

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz


# Perform evaluation, allowing objects to be matched between classes
results = dataset.evaluate_detections(
    "predictions",
    gt_field="detections",
    method="coco",
    classwise=True,
)

# Generate a confusion matrix for the specified classes
plot = results.plot_confusion_matrix(classes = classes)
plot.show()